In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("matchups1-1.csv",  nrows=999999)

In [4]:
df.head()

,P_MATCH_ID,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,GAMEVERSION,Unnamed: 13
0,BR1_2304032235_utility,5218,16,False,1,6,6,Bard,28,UNNl1KcPO98UoXiuRpQBefEKJbtCF_80b0_2s0Cwa5FiYi...,4500,batata 12121212,11.13.382.1241,NaN
1,BR1_2304032235_utility,7515,29,True,1,20,1,Blitzcrank,40,w2DLeo91qdfD72dpGgapMOKh_4IZ9IMF29neabiS0QTe8W...,7716,love yourseIf,11.13.382.1241,NaN
2,BR1_2304032235_jungle,9197,47,False,5,5,5,Nocturne,17,wDtmVguiopT93yrxtv2L88LxAVWC8E2fj_F3FDW81nCuSU...,9696,NTM HACKER,11.13.382.1241,NaN
3,BR1_2304032235_jungle,10564,37,True,6,8,4,Kayn,12,zVKtTZrdKVIpXwIMlsuSQjwOgqxx0DMhnWDFL7MrAKxXZq...,15291,unsuri,11.13.382.1241,NaN
4,BR1_2303451507_top,10598,158,False,6,8,7,Pantheon,17,sTevUOXxKjNW7dpbtyu9wjn8KZxzN63_f2MfGc1EALDjtq...,20568,Nome e Numeros,11.13.382.1241,NaN


In [5]:
df['matchID'] = df['P_MATCH_ID'].str.split('_').str[1]
df['position'] = df['P_MATCH_ID'].str.split('_').str[2]
df["WIN"] = df["WIN"].astype(int)
df.head()

,P_MATCH_ID,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,GAMEVERSION,Unnamed: 13,matchID,position
0,BR1_2304032235_utility,5218,16,0,1,6,6,Bard,28,UNNl1KcPO98UoXiuRpQBefEKJbtCF_80b0_2s0Cwa5FiYi...,4500,batata 12121212,11.13.382.1241,NaN,2304032235,utility
1,BR1_2304032235_utility,7515,29,1,1,20,1,Blitzcrank,40,w2DLeo91qdfD72dpGgapMOKh_4IZ9IMF29neabiS0QTe8W...,7716,love yourseIf,11.13.382.1241,NaN,2304032235,utility
2,BR1_2304032235_jungle,9197,47,0,5,5,5,Nocturne,17,wDtmVguiopT93yrxtv2L88LxAVWC8E2fj_F3FDW81nCuSU...,9696,NTM HACKER,11.13.382.1241,NaN,2304032235,jungle
3,BR1_2304032235_jungle,10564,37,1,6,8,4,Kayn,12,zVKtTZrdKVIpXwIMlsuSQjwOgqxx0DMhnWDFL7MrAKxXZq...,15291,unsuri,11.13.382.1241,NaN,2304032235,jungle
4,BR1_2303451507_top,10598,158,0,6,8,7,Pantheon,17,sTevUOXxKjNW7dpbtyu9wjn8KZxzN63_f2MfGc1EALDjtq...,20568,Nome e Numeros,11.13.382.1241,NaN,2303451507,top


In [6]:
df = df.groupby(['matchID', 'WIN'], as_index = False).sum()
df

C:\Users\oqiu1\AppData\Local\Temp\ipykernel_24468\1228642102.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['matchID', 'WIN'], as_index = False).sum()


,matchID,WIN,GOLDEARNED,TOTALMINIONSKILLED,KILLS,ASSISTS,DEATHS,VISIONSCORE,TOTALDAMAGEDEALTTOCHAMPIONS,Unnamed: 13
0,1001288357,0,31550,467,5,4,21,75,42125,0.0
1,1001288357,1,41019,526,21,24,5,107,51758,0.0
2,1001303651,0,46640,486,22,26,39,124,85366,0.0
3,1001303651,1,59825,520,39,76,22,135,95004,0.0
4,1001341779,0,51776,723,18,28,28,197,56590,0.0
...,...,...,...,...,...,...,...,...,...,...
10553,971925188,1,59431,632,28,50,23,124,88083,0.0
10554,971929106,0,51277,677,21,37,36,195,60394,0.0
10555,971929106,1,63935,691,36,54,21,199,76772,0.0
10556,971951103,0,18495,277,3,3,13,37,15352,0.0


# Individual team Perceptron

In [7]:
df2 = df.drop(columns=["matchID","Unnamed: 13"])
X = df2.drop(columns=["WIN"])
y = df2["WIN"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
p = Perceptron(random_state=42)
p.fit(X_train, y_train)

prediction_train = p.predict(X_train)
print("Train Prediction: ", prediction_train)

prediction_test = p.predict(X_test)
print("Test Prediction: ", prediction_test)

train_score = accuracy_score(prediction_train, y_train)
print("Train Score: ", train_score)

test_score = accuracy_score(prediction_test, y_test)
print("Test Score: ", test_score)

Train Prediction:  [1 0 1 ... 0 1 0]
Test Prediction:  [1 1 1 ... 1 1 1]
Train Score:  0.6974409727131344
Test Score:  0.7024390243902439


# Match Perceptron

In [8]:
df3 = df.drop(columns=["Unnamed: 13"])
df3.head()

,matchID,WIN,GOLDEARNED,TOTALMINIONSKILLED,KILLS,ASSISTS,DEATHS,VISIONSCORE,TOTALDAMAGEDEALTTOCHAMPIONS
0,1001288357,0,31550,467,5,4,21,75,42125
1,1001288357,1,41019,526,21,24,5,107,51758
2,1001303651,0,46640,486,22,26,39,124,85366
3,1001303651,1,59825,520,39,76,22,135,95004
4,1001341779,0,51776,723,18,28,28,197,56590


In [22]:
# create two separate dataframes for the winning and losing teams
win_df = df3[df3['WIN'] == 1]
lose_df = df3[df3['WIN'] == 0]

# merge the two dataframes by matchID, dropping the 'WIN' column from one of them
merged_df = pd.merge(win_df, lose_df.drop('WIN', axis=1), on='matchID')
merged_df1 = pd.merge(lose_df, win_df.drop('WIN', axis=1), on='matchID')

# rename columns to indicate whether they correspond to the winning or losing team
#merged_df.columns = [col + '_team1' if col != 'matchID' else col for col in merged_df.columns[:8]] + \
#                    [col + '_team2' for col in merged_df.columns[8:]]
#
# reorder columns to match the order you specified
finalMerged = pd.concat([merged_df, merged_df1], ignore_index = True)
finalMerged



,matchID,WIN,GOLDEARNED_x,TOTALMINIONSKILLED_x,KILLS_x,ASSISTS_x,DEATHS_x,VISIONSCORE_x,TOTALDAMAGEDEALTTOCHAMPIONS_x,GOLDEARNED_y,TOTALMINIONSKILLED_y,KILLS_y,ASSISTS_y,DEATHS_y,VISIONSCORE_y,TOTALDAMAGEDEALTTOCHAMPIONS_y
0,1001288357,1,41019,526,21,24,5,107,51758,31550,467,5,4,21,75,42125
1,1001303651,1,59825,520,39,76,22,135,95004,46640,486,22,26,39,124,85366
2,1001341779,1,58452,708,28,46,18,173,70753,51776,723,18,28,28,197,56590
3,1001406448,1,62365,516,50,58,35,181,104271,55554,569,35,49,50,147,109302
4,1001432370,1,51374,586,30,52,13,148,69060,40703,556,12,11,30,133,34896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10553,971856103,0,37860,482,15,13,35,113,49076,50975,499,35,59,16,125,69968
10554,971878861,0,31296,451,6,9,21,92,32934,40006,448,21,24,6,110,43102
10555,971925188,0,56201,704,23,44,28,184,95377,59431,632,28,50,23,124,88083
10556,971929106,0,51277,677,21,37,36,195,60394,63935,691,36,54,21,199,76772
